# Data exploration and preprocessing

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizerFast

from transformers import pipeline
import torch
import re
import os

from models.condbert import CondBertRewriter
from models.choosers import EmbeddingSimilarityChooser
from models.masked_token_predictor_bert import MaskedTokenPredictorBert


ModuleNotFoundError: No module named 'models'

In [1]:
import numpy as np

from flair.data import Sentence
from flair.embeddings import WordEmbeddings

Firstly, create and show image with half size in both axis with original ones.

In [8]:
df = pd.read_csv('filtered.tsv', sep='\t')
df = df.drop(['Unnamed: 0'], axis=1)

In [9]:
df.tail(5)
dataset = df.copy()
dataset = dataset.head(20000)
dataset

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348
...,...,...,...,...,...,...
19995,Now what else did the prostitute say?,what else did the bride say?,0.725349,0.236842,0.978869,0.000040
19996,I'm gonna spew.,I'll puke.,0.699393,0.312500,0.088549,0.978189
19997,"""He is that,"" said Estraven, ""but I asked too ...","""he's a traitor,"" Estraven told me, ""but I ask...",0.693783,0.061224,0.000046,0.964762
19998,"Hello, new dice smell.","hey, the smell of new tits.",0.647981,0.178571,0.000183,0.993284


In [10]:
aboba1 = np.mean(dataset["ref_tox"])
aboba2 = np.mean(dataset["trn_tox"])
print(aboba1, aboba2)

0.5416104048595838 0.43504418545919943


In [11]:
for i in dataset.index:
    if dataset.loc[i, 'trn_tox'] >= 0.5:
        # Swap 'reference' and 'translation'
        dataset.loc[i, 'reference'], dataset.loc[i, 'translation'] = dataset.loc[i, 'translation'], dataset.loc[i, 'reference']
        
        # Swap 'ref_tox' and 'trn_tox'
        dataset.loc[i, 'ref_tox'], dataset.loc[i, 'trn_tox'] = dataset.loc[i, 'trn_tox'], dataset.loc[i, 'ref_tox']

In [12]:
aboba1 = np.mean(dataset["ref_tox"])
aboba2 = np.mean(dataset["trn_tox"])
print(aboba1, aboba2)

0.9400417279332877 0.036612862385495604


In [13]:
texts = df["reference"].iloc[:20000].tolist()
labels = (df["ref_tox"].iloc[:20000] >= 0.5).astype(int).tolist()

In [146]:
print(texts[20:30])
labels[20:30]

['Fatuous Xerxes blithely surrendered control to the computer grid, ordering it to take care of whatever troubles might arise.', "That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.", 'Real life starts the first time you fuck, kid.', "I think you are the strangest man I've ever met.", "I say creepy, I mean, it's totally batshit crazy I can even talk to you.", "Shit, this one I can't even pronounce.", 'I like that shit.', "Trying to keep me fucking drugged so I don't know what's going on.", 'How is this not porn? This is porn that comes home.', 'Hey, leave the poor bastard alone!']


[0, 1, 1, 0, 0, 1, 1, 1, 1, 1]

In [14]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Assuming 'texts' is your list of texts and 'labels' is your list of toxicity labels
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Train the logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X, labels)

# Get the feature names (words) and their weights from the logistic regression model
feature_names = vectorizer.get_feature_names_out()
feature_weights = lr_model.coef_[0]

# Create a dictionary mapping words to their toxicity scores (weights)
word_toxicity_scores = dict(zip(feature_names, feature_weights))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lea

In [15]:
# Now for each sentence in 'texts', compute the toxicity score for each word and find the toxic words
toxic_words_in_sentences = []

for i in range(len(texts)):
    # Split the sentence into words, considering punctuation
    words = re.findall(r'\b\w+\b', texts[i])
    
    # Compute the toxicity scores for the words
    scores = [word_toxicity_scores.get(word, 0) for word in words]
    
    # Compute the threshold
    t = max(0.2, max(scores)/2)
    
    # Find the toxic words
    toxic_words = [word for word, score in zip(words, scores) if score > t]
    
    toxic_words_in_sentences.append(toxic_words)
toxic_words_in_sentences = [item for sublist in toxic_words_in_sentences for item in sublist]

In [16]:
word_toxicity_scores

{'00': -1.250372381536675,
 '000': -0.06775326951346249,
 '05': 0.0024472439507403645,
 '0ne': -0.06306058166157542,
 '0r': 0.30697074899817106,
 '0um': -0.10849973932604541,
 '10': 0.4457266956141903,
 '100': 0.19083089630205974,
 '1000': 0.002718057516265926,
 '106': 0.018801527289510542,
 '11': 0.016283183744113125,
 '111': -0.1666382048057594,
 '12': 0.24316221405171637,
 '125': 0.019140219920416864,
 '13': 0.3568961175276489,
 '14': -0.10330965650432955,
 '15': -0.15942082482478764,
 '150': -0.09298847216025222,
 '157': 0.364261145498049,
 '16': -0.6946483404891742,
 '160': 0.37501826313474396,
 '16th': -0.25716396324733704,
 '17': -0.18415811468507468,
 '18': 0.2723239852781559,
 '1820': -0.026815669192258248,
 '18th': -0.05486810958425707,
 '19': 0.02313261340121962,
 '1948': 0.21326929137296713,
 '1955': -0.047302450938045824,
 '1959': -0.07223563951093512,
 '1973': -0.1774444220131933,
 '1975': 0.06013112893421485,
 '1978': 0.13949746054756898,
 '1980': 0.16547441312593356,
 '

In [17]:
toxic_words_in_sentences

['waste',
 'nasty',
 'spare',
 'your',
 'put',
 'her',
 'die',
 'ass',
 'black',
 'hell',
 'crap',
 'want',
 'sweet',
 'kill',
 'him',
 'shit',
 'dog',
 'die',
 'die',
 'whatever',
 'kill',
 'fuck',
 'are',
 'met',
 'crazy',
 'shit',
 'fucking',
 'porn',
 'porn',
 'bastard',
 'want',
 'punching',
 'butt',
 'shit',
 'front',
 'him',
 'killer',
 'kill',
 'morals',
 'integrity',
 'killing',
 'savage',
 'fucking',
 'killer',
 'bitch',
 'your',
 'happens',
 'bored',
 'witless',
 'work',
 'these',
 'irresponsible',
 'how',
 'are',
 'die',
 'twisted',
 'dopey',
 'kill',
 'smelling',
 'she',
 'anal',
 'nuts',
 'burn',
 'girlfriends',
 'are',
 'dead',
 'bastard',
 'braver',
 'foolhardy',
 'hell',
 'nolan',
 'crap',
 'hope',
 'your',
 'pee',
 'pee',
 'send',
 'sex',
 'shit',
 'damn',
 'bloody',
 'rotten',
 'hell',
 'your',
 'womanizing',
 'your',
 'college',
 'dead',
 'fight',
 'want',
 'clearly',
 'crybaby',
 'niggers',
 'shit',
 'filthy',
 'fucking',
 'kill',
 'ass',
 'whore',
 'kill',
 'mothe

In [23]:
bimba = texts[20:30]

In [24]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
masked_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

for sentence in bimba:
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    # Identify the toxic words and replace them with [MASK]
    input_ids = inputs['input_ids'].tolist()[0]
    for i in range(len(input_ids)):
        if tokenizer.decode([input_ids[i]]) in toxic_words_in_sentences:
            input_ids[i] = tokenizer.mask_token_id

    # Convert back to tensors
    inputs['input_ids'] = torch.tensor([input_ids])

    # Predict the masked tokens
    outputs = model(**inputs)
    predictions = outputs.logits

    # Get the predicted tokens
    predicted_ids = torch.argmax(predictions, dim=2)

    # Decode the tokens to strings
    predicted_sentence = tokenizer.decode(predicted_ids[0])

    print(predicted_sentence)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


. contemptuous xerxes blithely surrendered control to the computer grid, allowing it to take care of whatever troubles might arise..
. that night, li'l had satisfied his thirst to die, though he knew she would never forgive him..
. " life starts the first time you die, too..
. i think you are the strangest man i. ve ever met..
. i mean creepy, i mean, it's totally batshit that i can even talk to you..
. well, this one i can't even pronounce..
. i like that one..
. trying to keep me completely drugged so i don't know what's going on..
. why is this not home? this is something that comes home..
. hey, leave the girl girl alone!.


In [137]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
masked_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# For each sentence
for i in range(len(bimba)):
    # Tokenize the sentence into words and punctuation
    tokens = re.findall(r"\b\w+\b|[\.,!?']", bimba[i])
    
    # Replace toxic words with [MASK]
    for j in range(len(tokens)):
        if tokens[j] in toxic_words_in_sentences:
            tokens[j] = '[MASK]'
    
    # Join the tokens back into a sentence
    bimba[i] = ' '.join(tokens)
    
    print(bimba[i])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fatuous Xerxes blithely surrendered control to the computer grid , [MASK] it to take [MASK] of [MASK] troubles might arise .
That night , Li ' l Dice satisfied his thirst to [MASK] , though he knew Shaggy would [MASK] forgive [MASK] .
Real life starts the first time you [MASK] , [MASK] .
I think you [MASK] the strangest man I ' ve ever [MASK] .
I [MASK] creepy , I mean , it ' s totally batshit [MASK] I can even talk to you .
Shit , this one I can ' t even pronounce .
I like that [MASK] .
Trying to keep me [MASK] drugged so I don ' t know what ' s [MASK] on .
How is this not [MASK] ? This is [MASK] that comes home .
Hey , leave the [MASK] [MASK] alone !


In [160]:
bimba = texts[20:30]

# Load pre-trained model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
masked_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Create a dictionary mapping tokens to their toxicity scores
token_toxicities = {tokenizer.encode(word, add_special_tokens=False)[0]: 1 for word in toxic_words_in_sentences}


# For each sentence in 'bimba'
for i in range(len(bimba)):
    # Tokenize the sentence into words and punctuation
    tokens = re.findall(r"\b\w+\b|[\.,!?']", bimba[i])
    
    # Identify toxic words
    toxic_word_indices = [j for j in range(len(tokens)) if tokens[j] in toxic_words_in_sentences]
    
    # Replace toxic words with [MASK] and preserve original tokens
    original_tokens = tokens.copy()
    for j in toxic_word_indices:
        tokens[j] = '[MASK]'
    
    # Join the tokens back into a sentence
    bimba[i] = ' '.join(tokens)
    
    print(bimba[i])
''' 
    # Predict replacements for '[MASK]'
    inputs = tokenizer(tokens, return_tensors='pt', padding=True, truncation=True)
    outputs = masked_model(**inputs)
    print(outputs)
    
    # Get predicted tokens and penalize tokens with positive toxicities
    logits = outputs.logits - token_toxicities * inputs['input_ids']
    preds = torch.argmax(logits, dim=-1)
    
    # Replace '[MASK]' with predicted tokens and rerank by similarity of embeddings
    for j in toxic_word_indices:
        inputs.input_ids[0][j] = preds[0][j]
    
    # Decode the tokens back into a sentence
    texts[i] = tokenizer.decode(inputs.input_ids[0])
# For each sentence in 'bimba'
for i in range(len(bimba)):
    # Tokenize the sentence into words and punctuation
    tokens = re.findall(r"\b\w+\b|[\.,!?']", bimba[i])
    
    # Identify toxic words
    toxic_word_indices = [j for j in range(len(tokens)) if tokens[j] in toxic_words_in_sentences]
    
    # Replace toxic words with [MASK] and preserve original tokens
    original_tokens = tokens.copy()
    for j in toxic_word_indices:
        tokens[j] = '[MASK]'
    
    # Join the tokens back into a sentence
    bimba[i] = ' '.join(tokens)
    
    print(bimba[i])
    
    # Predict replacements for '[MASK]'
    inputs = tokenizer(tokens, return_tensors='pt', padding=True, truncation=True)
    outputs = masked_model(**inputs)
    
    # Get predicted tokens and penalize tokens with positive toxicities
    toxicity_scores = torch.tensor([token_toxicities.get(token.item(), 0) for token in inputs['input_ids'][0]], dtype=torch.float)
    toxicity_scores = toxicity_scores.unsqueeze(0).unsqueeze(2)
    logits = outputs.logits - toxicity_scores
    preds = torch.argmax(logits, dim=-1)
    
    # Replace '[MASK]' with predicted tokens and rerank by similarity of embeddings
    mask_indices = (inputs.input_ids[0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
    for j, mask_index in enumerate(mask_indices):
        inputs.input_ids[0][mask_index] = preds[0][j]

    # Decode the tokens back into a sentence
    decoded_sentence = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
    print(decoded_sentence, 1743)
    '''

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Fatuous Xerxes blithely surrendered control to the computer grid , [MASK] it to take [MASK] of [MASK] troubles might arise .
That night , Li ' l Dice satisfied his thirst to [MASK] , though he knew Shaggy would [MASK] forgive [MASK] .
Real life starts the first time you [MASK] , [MASK] .
I think you [MASK] the strangest man I ' ve ever [MASK] .
I [MASK] creepy , I mean , it ' s totally batshit [MASK] I can even talk to you .
Shit , this one I can ' t even pronounce .
I like that [MASK] .
Trying to keep me [MASK] drugged so I don ' t know what ' s [MASK] on .
How is this not [MASK] ? This is [MASK] that comes home .
Hey , leave the [MASK] [MASK] alone !


' \n    # Predict replacements for \'[MASK]\'\n    inputs = tokenizer(tokens, return_tensors=\'pt\', padding=True, truncation=True)\n    outputs = masked_model(**inputs)\n    print(outputs)\n    \n    # Get predicted tokens and penalize tokens with positive toxicities\n    logits = outputs.logits - token_toxicities * inputs[\'input_ids\']\n    preds = torch.argmax(logits, dim=-1)\n    \n    # Replace \'[MASK]\' with predicted tokens and rerank by similarity of embeddings\n    for j in toxic_word_indices:\n        inputs.input_ids[0][j] = preds[0][j]\n    \n    # Decode the tokens back into a sentence\n    texts[i] = tokenizer.decode(inputs.input_ids[0])\n# For each sentence in \'bimba\'\nfor i in range(len(bimba)):\n    # Tokenize the sentence into words and punctuation\n    tokens = re.findall(r"\x08\\w+\x08|[\\.,!?\']", bimba[i])\n    \n    # Identify toxic words\n    toxic_word_indices = [j for j in range(len(tokens)) if tokens[j] in toxic_words_in_sentences]\n    \n    # Replace to

In [167]:
with open("bad-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))

with open("good-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))




In [168]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
masked_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def adjust_logits(logits, label=0):
    return logits - token_toxicities * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(masked_model, tokenizer, max_len=250, device=device, label=0,
                                     contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=masked_model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word_toxicity_scores,
    token_toxicities=word_toxicity_scores,
    predictor=predictor,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'MaskedTokenPredictorBert' is not defined

In [148]:
texts[20:30]

['Fatuous Xerxes blithely surrendered control to the computer grid, ordering it to take care of whatever troubles might arise.',
 "That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.",
 'Real life starts the first time you fuck, kid.',
 "I think you are the strangest man I've ever met.",
 "I say creepy, I mean, it's totally batshit crazy I can even talk to you.",
 "Shit, this one I can't even pronounce.",
 'I like that shit.',
 "Trying to keep me fucking drugged so I don't know what's going on.",
 'How is this not porn? This is porn that comes home.',
 'Hey, leave the poor bastard alone!']

In [149]:
bimba

['[CLS] fatuous [SEP] [PAD] [PAD]',
 '[CLS] that [SEP]',
 '[CLS] real [SEP]',
 '[CLS] i [SEP] [PAD]',
 '[CLS] i [SEP] [PAD]',
 '[CLS] shit [SEP] [PAD] [PAD]',
 '[CLS] i [SEP]',
 '[CLS] trying [SEP]',
 '[CLS] how [SEP]',
 '[CLS] hey [SEP]']